In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sys
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

sys.path.append('../')

In [6]:
from src.layers import (
    NumericalFeatureEmbedding,
    CategoricalFeatureEmbedding, 
    FeatureEmbedding, 
    TransformerEncoder, 
    MLP
)

### Numerical Features Embeddings

In [29]:

num_features = 2
dim_token = 6

x_num = np.random.random(size=(100, 2))

d_sqrt_inv = 1 / np.sqrt(dim_token)

# Learnable weights
w = np.random.uniform(low=-d_sqrt_inv, high=d_sqrt_inv, size=(num_features,dim_token))
b = np.random.uniform(low=-d_sqrt_inv, high=d_sqrt_inv, size=(num_features,dim_token))

output = w[np.newaxis] *x_num[...,np.newaxis] + b

print("weights shape: ",w[np.newaxis].shape)
print("x shape: ",x_num[...,np.newaxis].shape)
print("output shape: ",(w[np.newaxis] *x_num[...,np.newaxis]).shape)

weights shape:  (1, 2, 6)
x shape:  (100, 2, 1)
output shape:  (100, 2, 6)


In [36]:
from sklearn.datasets import make_classification

num_features = 10
dim_token = 32

X, y = make_classification(
    n_samples=1000, n_features=num_features, 
    n_informative=num_features, n_redundant=0, 
    n_repeated=0, n_classes=2, 
    n_clusters_per_class=2,
    shuffle=True, random_state=123
)

ne = NumericalFeatureEmbedding(num_features=num_features,dim_token=dim_token)

ne(X).shape

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)


TensorShape([1000, 10, 32])

In [72]:
# Basic MLP classification without NumericalFeatureEmbedding 
inputs = keras.layers.Input(shape=X.shape[1])
x = keras.layers.Dense(32, activation='relu')(inputs)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(16, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

base_model = keras.Model(inputs=inputs, outputs=outputs)

base_model.compile(loss='binary_crossentropy',metrics=['accuracy'], optimizer='adam')

base_history = base_model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=20)

base_result = base_model.evaluate(X_val, y_val)

Epoch 1/20
25/25 [==============================] - 1s 9ms/step - loss: 0.8012 - accuracy: 0.5375 - val_loss: 0.6844 - val_accuracy: 0.6050
Epoch 2/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5915 - accuracy: 0.6888 - val_loss: 0.6031 - val_accuracy: 0.6800
Epoch 3/20
25/25 [==============================] - 0s 4ms/step - loss: 0.5059 - accuracy: 0.7775 - val_loss: 0.5578 - val_accuracy: 0.7200
Epoch 4/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4638 - accuracy: 0.7962 - val_loss: 0.5272 - val_accuracy: 0.7400
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4216 - accuracy: 0.8263 - val_loss: 0.4973 - val_accuracy: 0.7600
Epoch 6/20
25/25 [==============================] - 0s 4ms/step - loss: 0.3818 - accuracy: 0.8462 - val_loss: 0.4690 - val_accuracy: 0.7850
Epoch 7/20
25/25 [==============================] - 0s 4ms/step - loss: 0.3617 - accuracy: 0.8537 - val_loss: 0.4478 - val_accuracy: 0.8000
Epoch 8/20
25/25 [==

In [91]:
# Basic MLP classification with NumericalFeatureEmbedding 
inputs = keras.layers.Input(shape=X.shape[1])
x = NumericalFeatureEmbedding(num_features=num_features,dim_token=dim_token)(inputs)
# x = keras.layers.Lambda(lambda x: tf.reduce_sum(x, axis=-2))(x)
x = keras.layers.Flatten(name="flatten_embeddings")(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(16, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

linear_model = keras.Model(inputs=inputs, outputs=outputs)

linear_model.compile(loss='binary_crossentropy',metrics=['accuracy'], optimizer='adam')

linear_history=linear_model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=20)

linear_result=linear_model.evaluate(X_val, y_val)

Epoch 1/20
25/25 [==============================] - 1s 15ms/step - loss: 0.5888 - accuracy: 0.7350 - val_loss: 0.5317 - val_accuracy: 0.7200
Epoch 2/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4259 - accuracy: 0.8238 - val_loss: 0.4847 - val_accuracy: 0.7350
Epoch 3/20
25/25 [==============================] - 0s 4ms/step - loss: 0.3668 - accuracy: 0.8475 - val_loss: 0.4287 - val_accuracy: 0.7850
Epoch 4/20
25/25 [==============================] - 0s 5ms/step - loss: 0.3375 - accuracy: 0.8487 - val_loss: 0.3961 - val_accuracy: 0.8100
Epoch 5/20
25/25 [==============================] - 0s 5ms/step - loss: 0.3076 - accuracy: 0.8737 - val_loss: 0.3583 - val_accuracy: 0.8300
Epoch 6/20
25/25 [==============================] - 0s 5ms/step - loss: 0.2850 - accuracy: 0.8750 - val_loss: 0.3285 - val_accuracy: 0.8650
Epoch 7/20
25/25 [==============================] - 0s 5ms/step - loss: 0.2681 - accuracy: 0.8963 - val_loss: 0.3025 - val_accuracy: 0.8850
Epoch 8/20
25/25 [=

In [92]:
inputs = keras.layers.Input(shape=X.shape[1])
x = NumericalFeatureEmbedding(num_features=num_features,dim_token=dim_token)(inputs)
x = TransformerEncoder(embed_dim=dim_token,num_heads=2,ff_dim=8)(x)
x = keras.layers.Flatten(name="flatten_embeddings")(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(16, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

transformer_model = keras.Model(inputs=inputs, outputs=outputs)

transformer_model.compile(loss='binary_crossentropy',metrics=['accuracy'], optimizer='adam')

transformer_history=transformer_model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=20)

transformer_result=transformer_model.evaluate(X_val, y_val)

Epoch 1/20
25/25 [==============================] - 2s 21ms/step - loss: 0.5691 - accuracy: 0.7113 - val_loss: 0.5292 - val_accuracy: 0.7300
Epoch 2/20
25/25 [==============================] - 0s 8ms/step - loss: 0.4223 - accuracy: 0.7987 - val_loss: 0.4516 - val_accuracy: 0.7800
Epoch 3/20
25/25 [==============================] - 0s 8ms/step - loss: 0.3618 - accuracy: 0.8400 - val_loss: 0.3863 - val_accuracy: 0.8400
Epoch 4/20
25/25 [==============================] - 0s 9ms/step - loss: 0.3043 - accuracy: 0.8737 - val_loss: 0.3732 - val_accuracy: 0.8300
Epoch 5/20
25/25 [==============================] - 0s 8ms/step - loss: 0.2819 - accuracy: 0.8875 - val_loss: 0.3307 - val_accuracy: 0.8500
Epoch 6/20
25/25 [==============================] - 0s 6ms/step - loss: 0.2648 - accuracy: 0.8975 - val_loss: 0.2921 - val_accuracy: 0.8800
Epoch 7/20
25/25 [==============================] - 0s 7ms/step - loss: 0.2445 - accuracy: 0.9013 - val_loss: 0.2908 - val_accuracy: 0.8750
Epoch 8/20
25/25 [=

In [97]:
print(f"Base results: epochs={len(base_history.history['loss'])} - loss={base_result[0]:.2f} - acc={base_result[1]:.2f}")
print(f"Linear Model results: epochs={len(linear_history.history['loss'])} - loss={linear_result[0]:.2f} - acc={linear_result[1]:.2f}")
print(f"Transformer Model results: epochs={len(transformer_history.history['loss'])} - loss={transformer_result[0]:.2f} - acc={transformer_result[1]:.2f}")


Base results: epochs=20 - loss=0.32 - acc=0.86
Linear Model results: epochs=20 - loss=0.26 - acc=0.90
Transformer Model results: epochs=20 - loss=0.24 - acc=0.92


### Categorical Features Embeddings

In [45]:
cardinalities = [5, 10, 20, 10,5]
num_features = len(cardinalities)
dim_token = 6

x_cat = np.concatenate([np.random.randint(low=0,high=c,size=(100,1)) for c in cardinalities], axis=1)

offsets = np.cumsum([0] + cardinalities[:-1], axis=0)

total_tokens = sum(cardinalities)

# # Learnable weights
d_sqrt_inv = 1 / np.sqrt(dim_token)

emb = keras.layers.Embedding(input_dim=total_tokens, output_dim=dim_token)
b = np.random.uniform(low=-d_sqrt_inv, high=d_sqrt_inv, size=(num_features,dim_token))
output = emb(x_cat + offsets) + b

print("input shape: ",x_num.shape)
print("output shape: ",output.shape)

input shape:  (100, 2)
output shape:  (100, 5, 6)


In [139]:

cardinalities = [3, 5, 10, 15, 20, 25, 30, 35]
num_features = len(cardinalities)
dim_token = 32

X, y = make_classification(
    n_samples=1000, n_features=num_features, 
    n_informative=num_features, n_redundant=0, 
    n_repeated=0, n_classes=2, 
    n_clusters_per_class=2,
    shuffle=True, random_state=123
)

# convert continuous to cat with quantils
X_cat = np.concatenate([
    np.digitize(X[:,i], bins=np.quantile(X[:,i], np.linspace(0,1,c+1))[1:-1])[...,np.newaxis]
    for i, c in enumerate(cardinalities)
], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X_cat, y, test_size=0.2)

ne = CategoricalFeatureEmbedding(cardinalities=cardinalities,dim_token=dim_token)

ne(X_cat).shape


TensorShape([1000, 8, 32])

In [142]:
# Basic MLP classification with CategoricalFeatureEmbedding 

inputs = keras.layers.Input(shape=X_train.shape[1], dtype='int64')

x = CategoricalFeatureEmbedding(cardinalities=cardinalities, dim_token=dim_token)(inputs)
# x = keras.layers.Lambda(lambda x: tf.reduce_sum(x, axis=-2))(x)
x = keras.layers.Flatten(name="flatten_embeddings")(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(16, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

linear_model = keras.Model(inputs=inputs, outputs=outputs)

linear_model.compile(loss='binary_crossentropy',metrics=['accuracy'], optimizer='adam')

base_history = linear_model.fit(x=X_train, y=y_train, validation_data=(X_val, y_val), epochs=20)

base_result = linear_model.evaluate(X_val, y_val)

Epoch 1/20
25/25 [==============================] - 1s 15ms/step - loss: 0.6779 - accuracy: 0.6125 - val_loss: 0.6516 - val_accuracy: 0.6900
Epoch 2/20
25/25 [==============================] - 0s 5ms/step - loss: 0.6283 - accuracy: 0.7175 - val_loss: 0.5839 - val_accuracy: 0.7500
Epoch 3/20
25/25 [==============================] - 0s 5ms/step - loss: 0.5562 - accuracy: 0.7613 - val_loss: 0.4931 - val_accuracy: 0.8050
Epoch 4/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4813 - accuracy: 0.7812 - val_loss: 0.4172 - val_accuracy: 0.8300
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4143 - accuracy: 0.8150 - val_loss: 0.3820 - val_accuracy: 0.8450
Epoch 6/20
25/25 [==============================] - 0s 6ms/step - loss: 0.3629 - accuracy: 0.8462 - val_loss: 0.3600 - val_accuracy: 0.8650
Epoch 7/20
25/25 [==============================] - 0s 5ms/step - loss: 0.3412 - accuracy: 0.8537 - val_loss: 0.3488 - val_accuracy: 0.8550
Epoch 8/20
25/25 [=